In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install libais
!{sys.executable} -m pip install bokeh


In [2]:
import re
import pandas as pd
import numpy as np
import time

import ais

from dask.distributed import Client
from dask import dataframe as ddf
from dask.multiprocessing import get
from multiprocessing import cpu_count

nCores = cpu_count()

start = time.time()
start

1585298064.2232609

In [3]:
def meta_parse(row):
    '''
    Parse meta strings. Either single or multiline headers:
    s:66,c:1555624754*3E
    or 
    g:1-2-0300,s:66,c:1555624791*46
    or 
    g:2-2-0300*5E
    and s:xx multiline messages?
    '''
    meta, meta_checksum = row['meta'].split("*")
    time_found = re.search(r'[0-9]{10}',meta)
    
    if time_found:
        event_time = time_found.group(0)
    else:
        event_time = None
    
    row['meta_checksum'] = meta_checksum
    row['meta_source'] = meta
    row['event_time'] = event_time
    
    return row


In [4]:
def ais_decode_to_dict(row, drop_message_type, multi = False):
    try:
        if row['frag_count'] == '1':
            decode_dict = ais.decode(row['payload'],int(row['padding']))
        else: 
            decode_dict = {}
    except Exception as err:
        "try padding to reach 70"
        try:
            decode_dict = ais.decode(row['payload'], 12)
#             print('It worked!')
        except Exception as err:
#             print('Still bad: {0}'.format(err))          
            decode_dict = {}
            
    if decode_dict.get('id') in drop_message_type:
        print('Dropping unwanted decoded message')
        decode_dict = {}
        
    row['decoded'] = decode_dict
    return row

In [5]:
# 170 Lines of AIS in the expected format:
# infile = 'full_test.log'

# 10% of the daily file. Around 70 megs/ 800K messages. 
# infile = 'decimate_test.log'

# Full day:
infile = 'TNPA_AIS.log.2019-04-19'

ais_sentence = ['packet',
  'frag_count',
  'frag_num',
  'seq_id',
  'radio_chan',
  'payload',
  'padding',
  'checksum']

first_cols = ['rx_time',
          'meta',
          'ais',
          'meta_checksum',
          'meta_source',
          'event_time',
          'decoded',
          'msg_num']

meta_cols = ['meta_checksum',
            'meta_source',
            'event_time']

drop_message_type = [7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27]


# Prepare Data

## Using Pandas

In [6]:
%%time

df = pd.read_csv(infile,
                 sep=r": \\|\\",
                 names=first_cols, 
                 header=None,
                 skiprows=1, 
                 skip_blank_lines=True,
                 engine="python")
 
 

CPU times: user 38.9 s, sys: 2.06 s, total: 40.9 s
Wall time: 40.9 s


In [27]:

# '!AIVDM,1,1,,,16<u3m0P24Mw@TUUd1ASEOwqqP00,0*55\n', 
ais.decode('16<u3m0P24Mw@TUUd1ASEOwqqP0016<u3m0P24Mw@TUUd1ASEOwqqP00', 2)

DecodeError: Ais1_2_3: AIS_ERR_BAD_BIT_COUNT

In [7]:
%%time

df[ais_sentence] = df['ais'].str.split(r',|\*', expand=True) 
df

CPU times: user 28.8 s, sys: 994 ms, total: 29.8 s
Wall time: 29.8 s


,rx_time,meta,ais,meta_checksum,meta_source,event_time,decoded,msg_num,packet,frag_count,frag_num,seq_id,radio_chan,payload,padding,checksum
0,"2019-04-19 00:00:00,774","s:66,c:1555624754*3E","!AIVDM,1,1,,,19NS36@02EMm;F5smJcsf9LF0@7u,0*3B",NaN,NaN,NaN,NaN,NaN,!AIVDM,1,1,,,19NS36@02EMm;F5smJcsf9LF0@7u,0,3B
1,"2019-04-19 00:00:00,775","s:66,c:1555624754*3E","!AIVDM,1,1,,,13csJ70vBOw2:u9s@l`23A`H087w,0*6F",NaN,NaN,NaN,NaN,NaN,!AIVDM,1,1,,,13csJ70vBOw2:u9s@l`23A`H087w,0,6F
2,"2019-04-19 00:00:00,777","s:66,c:1555624756*3C","!AIVDM,1,1,,,1815CD@01dNCOGee6Qdbt8pF0@8>,0*20",NaN,NaN,NaN,NaN,NaN,!AIVDM,1,1,,,1815CD@01dNCOGee6Qdbt8pF0@8>,0,20
3,"2019-04-19 00:00:00,778","s:66,c:1555624756*3C","!AIVDM,1,1,,,13UMff0vAoNj`EicjOASc2tJ0@8B,0*4C",NaN,NaN,NaN,NaN,NaN,!AIVDM,1,1,,,13UMff0vAoNj`EicjOASc2tJ0@8B,0,4C
4,"2019-04-19 00:00:00,779","s:66,c:1555624756*3C","!AIVDM,1,1,,,177=a8001RMpV93hSCO:vHlT0@8E,0*36",NaN,NaN,NaN,NaN,NaN,!AIVDM,1,1,,,177=a8001RMpV93hSCO:vHlT0@8E,0,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7992781,"2019-04-19 23:59:59,226","s:66,c:1555711162*3A","!AIVDM,1,1,,,1815DP@01oMpM7kgioMs18ln0D00,0*5C",NaN,NaN,NaN,NaN,NaN,!AIVDM,1,1,,,1815DP@01oMpM7kgioMs18ln0D00,0,5C
7992782,"2019-04-19 23:59:59,226","s:66,c:1555711163*3B","!AIVDM,1,1,,,177;A;0wR8O1bV9lG=HlkCnT0H<F,0*6B",NaN,NaN,NaN,NaN,NaN,!AIVDM,1,1,,,177;A;0wR8O1bV9lG=HlkCnT0H<F,0,6B
7992783,"2019-04-19 23:59:59,227","s:66,c:1555711163*3B","!AIVDM,1,1,,,15CN<T001l0EWM?b@`A3MBtb00S=,0*4C",NaN,NaN,NaN,NaN,NaN,!AIVDM,1,1,,,15CN<T001l0EWM?b@`A3MBtb00S=,0,4C
7992784,"2019-04-19 23:59:59,228","s:66,c:1555711163*3B","!AIVDM,1,1,,,1;23g`000;Ki8<kQciidr83420S;,0*5C",NaN,NaN,NaN,NaN,NaN,!AIVDM,1,1,,,1;23g`000;Ki8<kQciidr83420S;,0,5C


In [8]:
df_meta = df[['meta','meta_checksum','meta_source','event_time']]

In [9]:
df_to_decode = df[['frag_count','payload','padding','decoded']]
df_to_decode

,frag_count,payload,padding,decoded
0,1,19NS36@02EMm;F5smJcsf9LF0@7u,0,NaN
1,1,13csJ70vBOw2:u9s@l`23A`H087w,0,NaN
2,1,1815CD@01dNCOGee6Qdbt8pF0@8>,0,NaN
3,1,13UMff0vAoNj`EicjOASc2tJ0@8B,0,NaN
4,1,177=a8001RMpV93hSCO:vHlT0@8E,0,NaN
...,...,...,...,...
7992781,1,1815DP@01oMpM7kgioMs18ln0D00,0,NaN
7992782,1,177;A;0wR8O1bV9lG=HlkCnT0H<F,0,NaN
7992783,1,15CN<T001l0EWM?b@`A3MBtb00S=,0,NaN
7992784,1,1;23g`000;Ki8<kQciidr83420S;,0,NaN


## Using Dask

In [10]:
%%time
client = Client()
client

CPU times: user 248 ms, sys: 377 ms, total: 626 ms
Wall time: 1.19 s


Client Scheduler: tcp://127.0.0.1:36821 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 10 Memory: 23.11 GB


In [11]:
%%time 
 
dask_meta   = ddf.from_pandas(df_meta, npartitions=100*nCores)
dask_meta = dask_meta.apply(lambda x : meta_parse(x), axis=1, meta = dask_meta)

dask_decode = ddf.from_pandas(df_to_decode, npartitions=100*nCores)
dask_decode = dask_decode.apply(lambda x : ais_decode_to_dict(x, drop_message_type, multi = False), axis = 1, meta = dask_decode) 

CPU times: user 2.15 s, sys: 620 ms, total: 2.77 s
Wall time: 2.67 s


In [24]:
%%time

meta = dask_meta.compute()

CPU times: user 37.7 s, sys: 5.81 s, total: 43.5 s
Wall time: 7min 11s


In [18]:
%%time
 
decoded = dask_decode.compute()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.scheduler - ERROR - Couldn't gather keys {"('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 982)": ['tcp://127.0.0.1:36157'], 

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:36157'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 982)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:33715'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 485)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:36157'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 807)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:33715'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 402)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:44499'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 671)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:34177'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 795)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:34177'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 92)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:33715'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 449)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:44499'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 925)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:44499'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 733)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:33715'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 144)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:36157'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 471)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp:

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:42469'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 497)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:42469'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 13)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:33715'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 911)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:44499'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 239)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:33715'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 391)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:42469'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 65)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp:/

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:34177'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 106)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:34177'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 96)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:36157'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 47)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:42469'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 701)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:36157'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 880)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:34177'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 580)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp:/

...

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:38439'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 440)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:38439'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 663)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41729'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 280)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41729'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 409)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:38439'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 278)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41729'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 619)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:38439'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 541)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:38439'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 324)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41835'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 986)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41729'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 289)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:38439'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 50)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:38439'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 243)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp:

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:38439'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 501)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:38439'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 747)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41835'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 793)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41835'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 475)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41729'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 523)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41835'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 758)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41835'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 281)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41729'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 508)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41835'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 563)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41729'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 762)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41729'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 865)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:41729'], ('apply-dc47ce8c6a53a4233fdac9dcfc7a1650', 752)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp

KilledWorker: ("('from_pandas-f352c6ef5fefc8d3d0f448ec71016260', 711)", <Worker 'tcp://127.0.0.1:38439', name: 2, memory: 0, processing: 132>)

In [19]:
%%time
multi_ddf = dask_decode[dask_decode.frag_count == '2']
multi_df = multi_ddf.compute()

CPU times: user 1min 7s, sys: 9.09 s, total: 1min 17s
Wall time: 14min 7s


In [23]:
meta

NameError: name 'meta' is not defined

In [22]:
multi_df = multi_df.join(meta)

NameError: name 'meta' is not defined

# Seperate in single and multi-line messages

In [20]:
%%time

multi_df['prev_payload'] = multi_df['payload'].shift(1)
multi_df['prev_event_time'] = multi_df['event_time'].shift(1)
multi_df['payload'] = multi_df['prev_payload'] + multi_df['payload']

multi_df =  multi_df[multi_df['prev_event_time'].notnull() & multi_df['meta_source'].str.contains('g:2')]

KeyError: 'event_time'

In [16]:
%%time
 
multi_df = multi_df.apply(lambda x : ais_decode_to_dict(x, drop_message_type), axis=1) 

NameError: name 'multi_df' is not defined

In [17]:
end = time.time()
print("Duration to process: {0} mins".format(round((end-start)/60,2)))

Duration to process: 28.92 mins
